
# 📘 Week 1 – LangChain Core Foundations (General Track)

> Mục tiêu tuần này: nắm vững **Prompt → LLM → Output Parser → Chain**, làm chủ **LCEL** (LangChain Expression Language) và **streaming**.  
> Yêu cầu: Python 3.10+, `pip install langchain langchain-openai tiktoken pydantic`

**Gợi ý cấu hình**:
```bash
pip install -U langchain langchain-openai tiktoken pydantic
export OPENAI_API_KEY=sk-...
```



## Buổi 1 – Hiểu cơ chế hoạt động của LLM trong LangChain

**Bạn sẽ học**: `ChatOpenAI`, cấu trúc messages, `invoke()` và `stream()`.


In [ ]:
import os  
from dotenv import load_dotenv
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [ ]:

# ⚙️ Khởi tạo LLM (OpenAI) – dùng GPT-4o làm ví dụ
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
load_dotenv(dotenv_path='.env')

# Sử dụng biến môi trường OPENAI_API_KEY
llm = ChatOpenAI(
    model="gpt-5-mini",
    temperature=0.2
    )

resp = llm.invoke([
    SystemMessage(content='Bạn là trợ giảng python'),
    HumanMessage(content="giải thích str trong 1")])
print(resp.content)


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
from langchain.tools import tool
from langchain.agents import create_agent
@tool
def get_price(x: int) -> int:
    """Trả về bình phương của số x."""
    return x * x
model=ChatOpenAI(
    model='gpt-5-mini',
    temperature=0.2
)
agent=create_agent(
    model=model,
    tools=[get_price]
)
result=agent.invoke({    
        'messages':[{
            'role':'user',
            'content':"Tính bình phương của 2 và 4"
        }

    ]}
)
print(result)


In [ ]:
result['messages'][-1].content

In [ ]:

# 🟢 Streaming từng token (hiệu ứng gõ chữ)
from langchain_openai import ChatOpenAI

llm_stream = ChatOpenAI(model="gpt-5-mini", temperature=0.3)
stream = llm_stream.stream("Viết 1 đoạn mô tả 2 câu về mô hình LLM.")
for chunk in stream:
    print(chunk.content or "", end="", flush=True)
print()



### Bài tập nhỏ
1) Thử đổi `model` và `temperature` để quan sát khác biệt.  
2) Viết prompt yêu cầu trả về đúng **2 gạch đầu dòng**.



## Buổi 2 – PromptTemplate & Chain cơ bản (LCEL)

**Bạn sẽ học**: `ChatPromptTemplate`, LCEL `|` để ghép pipeline, tạo **chain 1 bước** và **nhiều bước**.


In [ ]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-5-mini")
prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là trợ lý nội dung, trả lời ngắn gọn, súc tích."),
    ("human", "Viết 1 slogan cho sản phẩm: {product} với tone {tone}.")
])

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"product": "Nước khoáng vị yuzu", "tone": "tinh tế"}))


In [ ]:
from langchain_core.load import dumpd, dumps
from IPython.display import JSON

# 1) Lấy dict “JSON-safe”
obj = dumpd(chain)

# 3) (tuỳ chọn) In ra chuỗi JSON có thụt dòng
text = dumps(chain, pretty=True)
print(text.encode('utf-8').decode('utf-8'))

In [ ]:

# 🔗 Nối nhiều bước bằng LCEL
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini-mini")

# Bước 1: sinh 5 slogan
p1 = ChatPromptTemplate.from_template(
    "Sinh 5 slogan ngắn cho sản phẩm {product}, tone {tone}. Chỉ liệt kê."
)
chain1 = p1 | llm | StrOutputParser()

# Bước 2: chọn 1 câu hay nhất
p2 = ChatPromptTemplate.from_template(
"""Chọn 1 câu hay nhất trong danh sách sau và giải thích ngắn lý do:
{candidates}"""
)
chain2 = p2 | llm | StrOutputParser()

candidates = chain1.invoke({"product": "fin68", "tone": "Chuyên nghiệp"})
final = chain2.invoke({"candidates": candidates})
print(final)


In [ ]:
# === Multi-stage Creative Chain (Fin68 style) ===
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# 0) Models
llm_gen = ChatOpenAI(model="gpt-4o-mini-mini", temperature=0.7)   # sinh slogan (sáng tạo)
llm_pick = ChatOpenAI(model="gpt-4o-mini-mini", temperature=0.0)  # chọn câu (ổn định)
llm_cap  = ChatOpenAI(model="gpt-4o-mini-mini", temperature=0.7)  # viết caption

# ------------------------------------------------
# (1) STAGE 1: Sinh 5 slogan
# ------------------------------------------------
p1 = ChatPromptTemplate.from_messages([
    ("system",
     "Bạn là chuyên gia marketing của Fin68. "
     "Viết ngắn gọn, súc tích, hiện đại và phù hợp kênh digital."),
    ("human",
     "Hãy viết 5 slogan cho sản phẩm {product} với tone {tone}. "
     "Chỉ liệt kê, mỗi dòng một slogan, không giải thích.")
])
gen_chain = p1 | llm_gen | StrOutputParser()

# ------------------------------------------------
# (2) STAGE 2: Chọn 1 câu hay nhất + lý do (Pydantic)
# ------------------------------------------------
class PickResult(BaseModel):
    best: str = Field(..., description="Slogan hay nhất, nguyên văn.")
    reason: str = Field(..., description="Lý do ngắn gọn vì sao chọn (<= 30 từ).")

# LLM trả thẳng PickResult (không cần JsonOutputParser)
llm_picker = llm_pick.with_structured_output(PickResult)

p2 = ChatPromptTemplate.from_messages([
    ("system",
     "Bạn là giám khảo sáng tạo của Fin68. "
     "Tiêu chí: rõ, độc đáo, dễ nhớ, hợp tone & sản phẩm."),
    ("assistant", "Danh sách đề cử:\n{candidates}"),
    ("human", "Hãy chọn 1 câu hay nhất và trả về JSON đúng schema.")
])
pick_chain = p2 | llm_picker

# ------------------------------------------------
# (3) STAGE 3: Viết caption quảng cáo 1 câu ngắn (<=120 ký tự)
# ------------------------------------------------
p3 = ChatPromptTemplate.from_messages([
    ("system",
     "Bạn là copywriter của Fin68. Viết caption quảng cáo 1 câu duy nhất, "
     "ngắn gọn, rõ ràng, hấp dẫn, tối đa 120 ký tự, có thể dùng emoji nhẹ nhàng."),
    ("assistant", "Slogan đã chọn: {best}"),
    ("human",
     "Viết caption cho sản phẩm {product} với tone {tone}. "
     "Chỉ trả lại 1 câu, không thêm gì khác.")
])
cap_chain = p3 | llm_cap | StrOutputParser()

# ------------------------------------------------
# DRIVER
# ------------------------------------------------
def run_campaign(product: str, tone: str):
    # (1) Sinh danh sách khẩu hiệu
    candidates = gen_chain.invoke({"product": product, "tone": tone})

    # (2) Chọn khẩu hiệu hay nhất (PickResult)
    picked: PickResult = pick_chain.invoke({"candidates": candidates})

    # (3) Viết caption dựa trên khẩu hiệu tốt nhất
    caption = cap_chain.invoke({
        "best": picked.best,
        "product": product,
        "tone": tone
    })

    return candidates, picked, caption

# Demo
if __name__ == "__main__":
    product = "Cà phê hòa tan 3in1"
    tone = "năng động"

    candidates, picked, caption = run_campaign(product, tone)

    print("=== (1) CÁC SLOGAN ĐỀ CỬ ===")
    print(candidates.strip())

    print("\n=== (2) SLOGAN TỐT NHẤT (JSON) ===")
    print(picked.model_dump_json(indent=2, ensure_ascii=False))

    print("\n=== (3) CAPTION 1 CÂU ===")
    print(caption.strip())


=== (1) CÁC SLOGAN ĐỀ CỬ ===
1. "Hòa tan vị ngon, khơi dậy năng lượng!"
2. "Cà phê 3in1 - Khởi đầu ngày mới tràn đầy hứng khởi!"
3. "Sảng khoái ngay lập tức, thưởng thức mọi lúc!"
4. "Năng động từng giọt, cà phê hòa tan tuyệt vời!"
5. "Pha trộn đam mê, thưởng thức hạnh phúc!"

=== (2) SLOGAN TỐT NHẤT (JSON) ===
{
  "best": "Cà phê 3in1 - Khởi đầu ngày mới tràn đầy hứng khởi!",
  "reason": "Dễ nhớ, thể hiện năng lượng tích cực cho ngày mới."
}

=== (3) CAPTION 1 CÂU ===
Khởi động ngày mới với Cà phê hòa tan 3in1 - Năng lượng tràn đầy chỉ trong 1 phút! ☕✨


In [ ]:
from operator import itemgetter
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Models
llm_gen = ChatOpenAI(model="gpt-4o-mini-mini", temperature=0.7)
llm_pick = ChatOpenAI(model="gpt-4o-mini-mini", temperature=0.0)
llm_cap  = ChatOpenAI(model="gpt-4o-mini-mini", temperature=0.7)

# ===== (1) Sinh 5 slogan =====
p1 = ChatPromptTemplate.from_messages([
    ("system", "Bạn là chuyên gia marketing Fin68, văn phong ngắn gọn, hiện đại."),
    ("human",  "Viết 5 slogan cho sản phẩm {product}, tone {tone}. Chỉ liệt kê.")
])
gen_chain = p1 | llm_gen | StrOutputParser()

# ===== (2) Chọn 1 câu hay nhất (Pydantic) =====
class PickResult(BaseModel):
    best: str = Field(..., description="Slogan hay nhất")
    reason: str = Field(..., description="Lý do ngắn gọn")

llm_picker = llm_pick.with_structured_output(PickResult)

p2 = ChatPromptTemplate.from_messages([
    ("system", "Bạn là giám khảo sáng tạo Fin68, đánh giá rõ–độc đáo–dễ nhớ."),
    ("assistant", "Danh sách đề cử:\n{candidates}"),
    ("human", "Chọn 1 câu hay nhất và trả về JSON đúng schema.")
])
pick_chain = p2 | llm_picker  # -> PickResult

# ===== (3) Viết caption <= 120 ký tự =====
p3 = ChatPromptTemplate.from_messages([
    ("system", "Bạn là copywriter Fin68: caption 1 câu, <=120 ký tự, có thể emoji."),
    ("assistant", "Slogan đã chọn: {best}"),
    ("human", "Viết caption cho {product}, tone {tone}. Chỉ trả lại 1 câu.")
])
cap_chain = p3 | llm_cap | StrOutputParser()

# ---- Helpers (biến các thao tác lấy trường thành Runnable) ----
get_product = RunnableLambda(itemgetter("product"))
get_tone    = RunnableLambda(itemgetter("tone"))

# Nếu 'picked' là Pydantic (khuyên dùng with_structured_output) → lấy .best
get_best_from_pydantic = RunnableLambda(lambda x: x["picked"].best)

# Nếu đôi khi bạn dùng dict thay vì Pydantic, xài bản fallback này:
# get_best_fallback = RunnableLambda(lambda x: x["picked"]["best"] if isinstance(x["picked"], dict) else x["picked"].best)

# ===== ONE-SHOT với RunnablePassthrough =====
one_shot_all = (
    RunnablePassthrough.assign(candidates=gen_chain)
    .assign(picked=pick_chain)
    .assign(
        caption=(
            {
                "best": get_best_from_pydantic,  # <-- bọc trong RunnableLambda
                "product": get_product,
                "tone": get_tone,
            } | cap_chain
        )
    )
)

one_shot_caption = (
    RunnablePassthrough.assign(candidates=gen_chain)
    .assign(picked=pick_chain)
    | {
        "best": get_best_from_pydantic,
        "product": get_product,
        "tone": get_tone,
    }
    | cap_chain
)

# ---- Demo
if __name__ == "__main__":
    payload = {"product": "Cà phê hòa tan 3in1", "tone": "năng động"}
    full = one_shot_all.invoke(payload)

    print(">>> INPUT:")
    print(payload)

    print("\n--- (1) CÁC SLOGAN ---")
    print(full["candidates"].strip())

    print("\n--- (2) PICKED (Pydantic) ---")
    print(full["picked"].model_dump_json(indent=2, ensure_ascii=False))

    print("\n--- (3) CAPTION ---")
    print(full["caption"].strip())

    print("\n>>> CAPTION ONLY:")
    print(one_shot_caption.invoke(payload).strip())


>>> INPUT:
{'product': 'Cà phê hòa tan 3in1', 'tone': 'năng động'}

--- (1) CÁC SLOGAN ---
1. "Khơi nguồn năng lượng, bắt đầu ngày mới!"
2. "Cà phê 3in1 – Giải pháp tiện lợi cho cuộc sống bận rộn!"
3. "Một ly, ba hương vị – Năng động từng giây!"
4. "Thức dậy cùng 3in1, tận hưởng trọn vẹn!"
5. "Cà phê nhanh chóng, phong cách sống hiện đại!"

--- (2) PICKED (Pydantic) ---
{
  "best": "Cà phê 3in1 – Giải pháp tiện lợi cho cuộc sống bận rộn!",
  "reason": "Câu slogan ngắn gọn, dễ nhớ và thể hiện rõ tính tiện lợi của sản phẩm, phù hợp với đối tượng khách hàng bận rộn."
}

--- (3) CAPTION ---
Khởi động ngày mới với cà phê 3in1 - Năng lượng dồi dào chỉ sau một tách! ☕💥

>>> CAPTION ONLY:
Khởi đầu ngày mới đầy năng lượng với cà phê hòa tan 3in1 – chỉ cần pha, khuấy và tận hưởng! ☕💪✨



### Bài tập nhỏ
- Biến chain trên thành **hàm** nhận `product, tone` → trả về `{best_line, reason}` ở dạng dict.
- Thêm bước 3: viết **biến thể** của slogan tốt nhất (3 biến thể).



## Buổi 3 – OutputParser: ép LLM trả kết quả có cấu trúc

**Bạn sẽ học**: `StrOutputParser`, `JsonOutputParser`, `PydanticOutputParser` và cách **validate** kết quả.


In [ ]:

# 🧱 JSON Output (khuyến nghị mô hình trả JSON)
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini-mini")
prompt = ChatPromptTemplate.from_template(
    "Trả lời ở dạng JSON với khóa: title, bullets (list).\n"
    "Viết nội dung giới thiệu ngắn cho chủ đề: {topic}."
)

raw = (prompt | llm | StrOutputParser()).invoke({"topic": "Nguyên lý hoạt động của LLM"})
print(raw)


```json
{
    "title": "Nguyên lý hoạt động của LLM",
    "bullets": [
        "Mô hình ngôn ngữ lớn (Large Language Models - LLM) là các mô hình trí tuệ nhân tạo được thiết kế để xử lý và hiểu ngôn ngữ tự nhiên.",
        "LLM sử dụng mạng nơ-ron sâu, cụ thể là các kiến trúc Transformer, để học từ lượng dữ liệu văn bản khổng lồ.",
        "Các mô hình này học cách dự đoán từ tiếp theo trong một câu, giúp chúng có khả năng sinh văn bản tự động, trả lời câu hỏi và thực hiện các tác vụ ngôn ngữ khác.",
        "Quá trình huấn luyện của LLM bao gồm hai giai đoạn chính: tiền huấn luyện và tinh chỉnh. Trong đó, tiền huấn luyện giúp mô hình học hiểu biết chung về ngôn ngữ, còn tinh chỉnh điều chỉnh mô hình cho các tác vụ cụ thể.",
        "Nhờ khả năng xử lý ngôn ngữ tự nhiên và sinh văn bản linh hoạt, các LLM ngày càng được ứng dụng rộng rãi trong nhiều lĩnh vực như dịch máy, tư vấn tự động, và phân tích ngữ nghĩa."
    ]
}
```


In [ ]:

# ✅ PydanticOutputParser – ép schema chặt chẽ
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

class Intro(BaseModel):
    title: str = Field(..., description="Tiêu đề ngắn gọn")
    bullets: list[str] = Field(..., description="Các ý chính")

parser = PydanticOutputParser(pydantic_object=Intro)

format_instructions = parser.get_format_instructions()
prompt = ChatPromptTemplate.from_template(
    "Hãy viết phần giới thiệu về {topic}.\n{format_instructions}"
).partial(format_instructions=format_instructions)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
out = (prompt | llm | parser).invoke({"topic":"LangChain cơ bản"})
print(out)
print(type(out))


title='Giới thiệu về LangChain' bullets=['LangChain là một framework mã nguồn mở.', 'Hỗ trợ phát triển các ứng dụng sử dụng mô hình ngôn ngữ lớn (LLMs).', 'Cung cấp các công cụ để quản lý chuỗi hội thoại và tích hợp với các nguồn dữ liệu khác nhau.', 'Hỗ trợ các tính năng như quản lý trạng thái, bộ nhớ và tích hợp API.', 'Thích hợp cho việc xây dựng các ứng dụng chatbot, trợ lý ảo và các hệ thống tự động hóa ngôn ngữ.']
<class '__main__.Intro'>


In [43]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "Tiêu đề ngắn gọn", "title": "Title", "type": "string"}, "bullets": {"description": "Các ý chính", "items": {"type": "string"}, "title": "Bullets", "type": "array"}}, "required": ["title", "bullets"]}\n```'


### Bài tập nhỏ
- Thêm trường `reading_time_min: int` vào schema và yêu cầu LLM ước lượng thời gian đọc.  
- Bắt lỗi khi LLM trả sai định dạng và **retry** với hướng dẫn bổ sung.



## Buổi 4 – Runnable, song song & streaming

**Bạn sẽ học**: `RunnableLambda`, `RunnableParallel`, stream kết quả LCEL.


In [ ]:

# 🧠 RunnableLambda – chuyển đổi dữ liệu trong pipeline
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_template(
    "Viết 3 gạch đầu dòng súc tích về: {topic}"
)

def uppercase(x: str) -> str:
    return x.upper()

chain = prompt | llm | StrOutputParser() | RunnableLambda(uppercase)
print(chain.invoke({"topic": "Quy trình học LangChain"}))


In [ ]:

# ⚡ RunnableParallel – chạy nhiều prompt cùng lúc và gom kết quả
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini")
p1 = ChatPromptTemplate.from_template("Đưa 1 phép ẩn dụ về {topic}.")
p2 = ChatPromptTemplate.from_template("Đưa 1 ví dụ thực tế về {topic}.")
p3 = ChatPromptTemplate.from_template("Cảnh báo 1 sai lầm thường gặp khi học {topic}.")

parallel = RunnableParallel(
    metaphor=(p1 | llm | StrOutputParser()),
    example=(p2 | llm | StrOutputParser()),
    pitfall=(p3 | llm | StrOutputParser())
)

result = parallel.invoke({"topic":"LangChain"})
result


In [ ]:

# 🟣 Streaming qua LCEL
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("Viết đoạn giới thiệu 2 câu về {topic}.")

chain = prompt | llm | StrOutputParser()
for token in chain.stream({"topic": "Output Parser trong LangChain"}):
    print(token, end="", flush=True)
print()



---

## Ôn tập nhanh & Gợi ý luyện tập
- **Luồng chuẩn**: Prompt → LLM → Parser → Chain (LCEL `|`).  
- **Parser**: ép định dạng **JSON/Pydantic** để máy đọc được.  
- **Runnable**: linh hoạt xử lý dữ liệu, chạy song song, stream.

### Bài tập cuối tuần
1) Tạo **pipeline 3 bước**: sinh dàn ý → viết đoạn → rút gọn về 120 ký tự.  
2) Viết **parser Pydantic** cho kết quả gồm `{title, summary, tags[]}`.  
3) Áp dụng **streaming** cho bước rút gọn để hiện “đang gõ”.

> Tuần sau: **Agent, Tool & Memory** rồi tiến dần sang **LangGraph & Multi-Agent**.
